<a href="https://colab.research.google.com/github/yinon2592/DL_Project_046211/blob/yinon_branch/section_a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!mkdir -p data
!wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/investigating-sentiment-analysis/data/training.1600000.processed.noemoticon.csv.zip -P data
!unzip -n -d data data/training.1600000.processed.noemoticon.csv.zip
!pip install transformers

--2023-06-22 16:35:18--  https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/investigating-sentiment-analysis/data/training.1600000.processed.noemoticon.csv.zip
Resolving nyc3.digitaloceanspaces.com (nyc3.digitaloceanspaces.com)... 162.243.189.2
Connecting to nyc3.digitaloceanspaces.com (nyc3.digitaloceanspaces.com)|162.243.189.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85088192 (81M) [application/zip]
Saving to: ‘data/training.1600000.processed.noemoticon.csv.zip’

training.1600000.pr 100%[===================>]  81.15M  80.0MB/s    in 1.0s    

2023-06-22 16:35:20 (80.0 MB/s) - ‘data/training.1600000.processed.noemoticon.csv.zip’ saved [85088192/85088192]

Archive:  data/training.1600000.processed.noemoticon.csv.zip
  inflating: data/training.1600000.processed.noemoticon.csv  
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 59.7 MB/s eta 0:00:

In [36]:
import pandas as pd
import re
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
# Step 1: Dataset Preparation
file_path="data/training.1600000.processed.noemoticon.csv"
df = pd.read_csv(file_path, encoding='ISO-8859-1', header=None)
df = df[[0, 5]]
df.columns = ['label', 'text']

df = df.sample(100, random_state=1)

# # df['label'] = df['label'].replace({0: 'negative', 2: 'neutral', 4: 'positive'})
# df['label'] = df['label'].replace({0: 'negative', 2: 'neutral', 4: 1})
# # Drop the rows with 'neutral' sentiment
# df = df[df['label'] != 'neutral']
df['label'] = df['label'].replace({4: 1})
df = df[df['label'] != 2]
print(df.label.value_counts())
print(df.sample(10))

# Step 2: Data Preprocessing
def clean_text(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)  # Remove usernames
    text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)  # Remove URLs
    text = re.sub(r'[^A-Za-z0-9]+', ' ', text)  # Remove special characters
    # Remove newlines and multiple whitespaces
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    # Remove special characters and punctuations
    text = re.sub(r'[^\w\s]', '', text)

    text = text.lower().strip()
    return text

df['text'] = df['text'].apply(clean_text)



0    57
1    43
Name: label, dtype: int64
         label                                               text
1136955      1                             Movies. Gonna see up. 
716696       0  @karenbyrne I am watching #churchonline, but I...
453627       0                   @track7grrl It's not on my CBS. 
914955       1  Please remember you need a helmet for the bike...
982623       1  @phoenixritu  thn u hv another option... come ...
902426       1     @jocalling hope our wishes come true someday. 
861927       1  jason mccartney is an absolute legend and such...
948001       1  @krystynzilla honestly, you're amazing.  thank...
639291       0  my fingers are like PRUNES. waaaaa  i hate tha...
1035799      1  I do not now nor ever will, trust the structur...


In [44]:
# Step 3: Split into Train, Validation, and Test Sets
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer
import textwrap

copy_df = df.head(5).copy()

def tokenize_text(text):
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2', truncation=True, padding=True, max_length=3)
    # tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    # tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    # Keys of the returned dictionary will be added to the dataset as columns
    return tokenizer.encode(text)

copy_df['text'] = copy_df['text'].apply(tokenize_text)

# wrapped_text = textwrap.fill(copy_df.sample(1).iloc(0), width=80)
print(copy_df.head(2)['text'])
# print(wrapped_text)

# train_ratio = 0.8
# val_ratio = 0.1
# test_ratio = 0.1

# # Split into train, validation, and test sets
# train_df, test_df = train_test_split(df, test_size=test_ratio, random_state=42)
# train_df, val_df = train_test_split(train_df, test_size=val_ratio/(train_ratio+val_ratio), random_state=42)

# print("Train set size:", len(train_df))
# print("Validation set size:", len(val_df))
# print("Test set size:", len(test_df))

514293    [72, 2051, 299, 1134, 4106, 14364, 14364, 1541...
142282    [568, 1312, 550, 257, 4320, 938, 1755, 1312, 3...
Name: text, dtype: object


In [5]:
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer, AdamW
import torch

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2ForSequenceClassification.from_pretrained('gpt2')

# Add a classification layer on top
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available
model.to(device)

# Freeze initial layers
for param in model.parameters():
    param.requires_grad = False

# Optimizer and loss function
optimizer = AdamW(model.parameters())
loss_fn = torch.nn.CrossEntropyLoss()

from torch.utils.data import DataLoader, Dataset

# Step 3: Tokenization and Dataset Creation
class SentimentDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        self.tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]['text']
        label = self.data.iloc[idx]['label']
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=128,  # Set your desired maximum sequence length
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': torch.tensor(1 if label == 'positive' else 0)  # Convert label to binary format
        }




# Create the datasets
train_dataset = SentimentDataset(train_df)
val_dataset = SentimentDataset(val_df)
test_dataset = SentimentDataset(test_df)


# Define batch size and create the DataLoader
batch_size = 8  # Set your desired batch size
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)



num_epochs = 5

# Fine-tuning loop
for epoch in range(num_epochs):
  model.train()
  for batch in train_dataloader:
      optimizer.zero_grad()
      input_ids = [item['input_ids'] for item in batch]
      attention_masks = [item['attention_mask'] for item in batch]
      labels = torch.tensor([item['labels'] for item in batch]).to(device)

      inputs = tokenizer.batch_encode_plus(
          input_ids,
          max_length=128,
          padding='longest',
          truncation=True,
          return_tensors='pt'
      )
      input_ids = inputs['input_ids'].to(device)
      attention_masks = inputs['attention_mask'].to(device)

      outputs = model(input_ids=input_ids, attention_mask=attention_masks, labels=labels)
      loss = outputs.loss
      loss.backward()
      optimizer.step()

  # # Validation and early stopping logic
  # model.eval()
  # # Perform validation and monitor performance
  # # Implement early stopping if needed

KeyboardInterrupt: ignored